In [1]:
#Python Language Version 

from platform import python_version
print('Versão da Linguagem Python Usada neste projeto:', python_version())

Versão da Linguagem Python Usada neste projeto: 3.8.5


# Análise Estatística e Modelagem Preditiva de Séries Temporais - Em Python

### Projeto - Qual o Efeito da Legalização da Maconha na Taxa de Criminalidade ao Longo do Tempo?

![Rodolfo Terra](https://raw.githubusercontent.com/rodolffoterra/Projct_Marijuana_Legalization_TS/main/imagnes/layout.png)

O projeto elaborado está longe de demonstrar opinião pessoal do cientista de dados.


### Definição do Problema

Em 2016, os eleitores da Califórnia nos EUA aprovaram a Proposição 64, que legalizou o uso recreativo da maconha no estado. Neste link você encontra os detalhes sobre a aprovação da Proposição 64:

<a href="https://ballotpedia.org/California_Proposition_64,_Marijuana_Legalization_(2016)">California Proposition 64, Marijuana Legalization (2016)</a>

Os opositores à medida apresentaram cinco principais objeções à mudança no Guia Oficial de Informações do Eleitor do estado. Eles argumentaram que a legalização:

- (1) Dobraria o número de mortes nas rodovias.

- (2) Permitiria o cultivo de maconha perto de escolas e parques.

- (3) Aumentaria a atividade do mercado negro e do cartel.

- (4) Prejudicaria as comunidades pobres com problemas de dependência por meio do influxo de novos pontos de vendas de maconha.

- (5) Aumentaria a criminalidade ao longo do tempo, especialmente nas áreas próximas aos pontos de venda de maconha legalizada (cada ponto de venda de maconha legalizada é chamado de marijuana dispensary). 

Nosso trabalho é "simples".

Vamos realizar uma análise de dados com base em séries temporais e análises geoespaciais, além de outras técnicas exploratórias mais gerais, para examinar se essas previsões se tornaram realidade em Los Angeles desde que a legalização entrou em vigor. Nosso foco principal será nos dados disponíveis sobre detenções por crimes relacionados à maconha e na taxa geral de crimes nas proximidades de dispensários. Vamos focar principalmente nos itens 2, 4 e 5 acima.

Os dados usados neste projeto estão disponíveis publicamente e são oferecidos pelo portal de dados abertos do governo de Los Angeles.

Você é favor ou contra à legalização da Maconha? Acompanhe o projeto passo a passo, leia atentamente cada comentário e então emita sua opinião com base em dados.

In [6]:
# The new versions of Pandas and Matplotlib bring several warning messages to the developer.; Let's disable this
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

# Imports for data Manipulation 
import re
import json
import time
import numpy as np
import pandas as pd
from yelp.client import Client
from yelpapi import YelpAPI

In [7]:
# Here you must include your Yelp API
minha_api = "G-7tkyXV3PObpunUwPENqR7R6aQJMKofQHsknJgwW5kJtjMMHSvuKTumtMjygMBpX2FewA9UaHZbFm6sIDUOOEoIAH7YLfZrShfTg8sk_D8e_Ys4XlADIKYmeyn-X3Yx"

In [8]:
# With the API defined, create ana ccess cliente
cliente_acesso = Client(minha_api)

A função abaixo é para formatar o arquivo retornado com as consultas da API e gerar um arquivo de log da execução (uma espécie de histórico).

In [9]:
# Função para formatar o arquivo com as consultas à API
def formata_arquivo(file_path, 
                    logfile = './dados/arquivo_log.txt',  
                    file_description = None): 
   
    # Com o nome completo do arquivo, aplicamos expressões regulares para a limpeza
    # Aqui ajustamos a extensão do arquivo
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start() 
    except:
        raise NameError('Digite um caminho de onde o arquivo pode ser encontrado.') 
    
    # Com o nome completo do arquivo, aplicamos expressões regulares para a limpeza
    # Aqui ajustamos timestamp para o nome do arquivo
    try:
        stamp = re.search('(?<!^)(?<!\.)[a-z]+_[a-z]+(?=\.)', file_path).start()
    except:
        raise NameError('Digite um caminho de onde o arquivo pode ser encontrado.') 
        
    # Formata o nome do arquivo adicionando o timestamp
    formatted_name = f'{file_path[:stamp]}{round(time.time())}_{file_path[stamp:]}' 
    
    # Se não tiver descrição no arquivo, adicionamos uma
    if not file_description:
        file_description = f'Arquivo gerado em: {time.asctime(time.gmtime(round(time.time())))}'
        
    # Abrimos o arquivo de log e gravamos o arquivo de dados formatado e a descrição
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')
        
    # Retornamos o arquivo de dados formatado e a descrição
    return formatted_name, file_description

A próxima função vai conectar ao Yelp com via API e retornar 1000 amostras, ou seja, 1000 registros de lojas que estão classificadas como "dispensaries" (lojas que vendem maconha legalizada em Los Angeles).

In [10]:
# Função de busca das lojas que vendem maconha legalizada em Los Angeles
def busca_yelp(category, 
               location, 
               offset_number = 0, 
               n_samples = 1000):
    
    # API
    yelp_api = YelpAPI(minha_api)
    
    # Registro do último resultado
    last_result = round(time.time())
    
    # Lista para gravar os resultados
    resultados = []
    
    # Tamanho
    size = 50
    
    # Inicializa o número de loops
    loops = 0
    
    # Inicializa o número de execuções
    run = 1
    
    # Inicializa o offset
    offset_count = offset_number
   
    # Loop para retonar os dados
    while loops < n_samples:
        
        print(f'Iniciando consulta {run}')
        
        # Fazendo a consulta
        posts = yelp_api.search_query(categories = category, 
                                      location = location, 
                                      offset = offset_count, 
                                      limit = size) 
            
        # Posts ligados a business
        resultados.extend(posts['businesses'])
        
        # Incrementa o número de loops
        loops += size
        
        # Incrementa o offset
        offset_count += 50
        
        # Aguarda 3 segundos para executar a próxima consulta
        time.sleep(3) 
        
        # Incrementa o número de execuções
        run += 1
       
    # Finalizado o loop, vamos retornar o nome do arquivo formatado e a descrição
    formatted_name, file_description = formata_arquivo(file_path =f'./dados/arquivo_{category}.json')
    
    # Abrimos o arquivo formatado e fazemos o dump (gravação) dos resultados da consulta em formato JSON
    with open(formatted_name, 'w+') as f:
        json.dump(resultados, f)
    
    print(f'\nConsulta concluída. Encontramos esse número de lojas: {len(resultados)} {category}s.')
    
    return print(f'\nEsse foi o último timestamp {round(time.time())}.')

In [11]:
# We perform the search with the type of establishment and the city
busca_yelp("cannabisdispensaries",'los angeles', n_samples = 1000)

Iniciando consulta 1
Iniciando consulta 2
Iniciando consulta 3
Iniciando consulta 4
Iniciando consulta 5
Iniciando consulta 6
Iniciando consulta 7
Iniciando consulta 8
Iniciando consulta 9
Iniciando consulta 10
Iniciando consulta 11
Iniciando consulta 12
Iniciando consulta 13
Iniciando consulta 14
Iniciando consulta 15
Iniciando consulta 16
Iniciando consulta 17
Iniciando consulta 18
Iniciando consulta 19
Iniciando consulta 20

Consulta concluída. Encontramos esse número de lojas: 326 cannabisdispensariess.

Esse foi o último timestamp 1610492792.


Observe na última linha acima o número do timestamp. Esse código representa o momento exato (hora, minuto, segundo) que o arquivo foi gerado. Pegamos o código e incluímos como prefixo no nome do arquivo na célula abaixo.

Isso tem que ser feito cada vez que executar este Jupyter Notebook. Mas é claro que você pode automatizar isso de outra forma, se desejar.

In [12]:
# Opens the JSON file for reading and generating the final list
with open(f'dados/1610492792_arquivo_cannabisdispensaries.json', 'r') as f:
    lista_lojas_la = json.load(f)

In [13]:
# Let's view a record of the list
lista_lojas_la[5]

{'id': 'qCrSlf_TSVxITOYIBXGdJQ',
 'alias': 'cannabis-cafe-west-hollywood',
 'name': 'Cannabis Cafe',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/CaA-BUOLFHxtnS7H3rs4wA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/cannabis-cafe-west-hollywood?adjust_creative=ZWy0ubD0XpxR_wgB9r9oJA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZWy0ubD0XpxR_wgB9r9oJA',
 'review_count': 677,
 'categories': [{'alias': 'cannabisdispensaries',
   'title': 'Cannabis Dispensaries'},
  {'alias': 'cafes', 'title': 'Cafes'}],
 'rating': 3.5,
 'coordinates': {'latitude': 34.092688, 'longitude': -118.344421},
 'transactions': ['delivery'],
 'location': {'address1': '1201 N La Brea Ave',
  'address2': '',
  'address3': None,
  'city': 'West Hollywood',
  'zip_code': '90038',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1201 N La Brea Ave', 'West Hollywood, CA 90038']},
 'phone': '+13239757676',
 'display_phone': '(323) 975-7676',
 'distance': 4057.181

Dados extraídos com sucesso! Vamos organizar e limpar esses dados.

In [19]:
# The function will organize the store list data on a pandas dataframe
def organiza_dados(lista_lojas, df):
    
    # Conert the list of store to a panda dataframe
    df = pd.DataFrame(lista_lojas)
    
    # List with column names that interest us 
    col_list = ['name',
                'is_closed',
                'url',
                'rating',
                'coordinates',
                'location',
                'price',
                'review_count']
    
    # Difines the list of columns above as the name of each column
    df = df[col_list]
    
    return df
    

In [20]:
# Apply the function and create our dataframe
df_lojas = organiza_dados(lista_lojas_la, df = 'df_lojas')

In [21]:
# Shape 
df_lojas.shape

(326, 8)

In [22]:
# We preview a sample of data
df_lojas.head()

,name,is_closed,url,rating,coordinates,location,price,review_count
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,"{'latitude': 34.08235, 'longitude': -118.272037}","{'address1': '2815 W Sunset Blvd', 'address2':...",NaN,299
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,"{'latitude': 34.08853, 'longitude': -118.3446}","{'address1': '979 N La Brea Ave', 'address2': ...",$$,289
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,"{'latitude': 34.0446, 'longitude': -118.25444}","{'address1': '735 S Broadway', 'address2': '',...",$$,344
3,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,"{'latitude': 34.0434989929199, 'longitude': -1...","{'address1': '', 'address2': None, 'address3':...",$$,118
4,Extra Special Delivery,False,https://www.yelp.com/biz/extra-special-deliver...,4.5,"{'latitude': 34.167, 'longitude': -118.37364}","{'address1': '', 'address2': None, 'address3':...",$,130


Vamos extrair a lista de latitudes e longitudes de cada loja, pois usaremos geolocalização mais tarde.

In [23]:
# Extract the list of latitudes and longitudes
lista_latitude = [lista_lojas_la[i]['coordinates']['latitude'] for i in range(len(lista_lojas_la))]
lista_longitude = [lista_lojas_la[i]['coordinates']['longitude'] for i in range(len(lista_lojas_la))]

E adicionamos em nosso dataframe.

In [24]:
df_lojas['latitude'] = lista_latitude
df_lojas['longitude'] = lista_longitude

In [25]:
# We preview a sample of data
df_lojas.head()

,name,is_closed,url,rating,coordinates,location,price,review_count,latitude,longitude
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,"{'latitude': 34.08235, 'longitude': -118.272037}","{'address1': '2815 W Sunset Blvd', 'address2':...",NaN,299,34.082350,-118.272037
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,"{'latitude': 34.08853, 'longitude': -118.3446}","{'address1': '979 N La Brea Ave', 'address2': ...",$$,289,34.088530,-118.344600
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,"{'latitude': 34.0446, 'longitude': -118.25444}","{'address1': '735 S Broadway', 'address2': '',...",$$,344,34.044600,-118.254440
3,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,"{'latitude': 34.0434989929199, 'longitude': -1...","{'address1': '', 'address2': None, 'address3':...",$$,118,34.043499,-118.250206
4,Extra Special Delivery,False,https://www.yelp.com/biz/extra-special-deliver...,4.5,"{'latitude': 34.167, 'longitude': -118.37364}","{'address1': '', 'address2': None, 'address3':...",$,130,34.167000,-118.373640


In [28]:
# With latitude and longitude, the location column with the full address is no longer needed and we remove it from the dataframe
df_lojas.drop('location', axis = 1, inplace = True)

In [29]:
# And then we create new location column containing a tuple with and longitude
df_lojas['location'] = list(zip(df_lojas['latitude'], df_lojas['longitude']))

In [30]:
# We preview a sample of data
df_lojas.head()

,name,is_closed,url,rating,coordinates,price,review_count,latitude,longitude,location
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,"{'latitude': 34.08235, 'longitude': -118.272037}",NaN,299,34.082350,-118.272037,"(34.08235, -118.272037)"
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,"{'latitude': 34.08853, 'longitude': -118.3446}",$$,289,34.088530,-118.344600,"(34.08853, -118.3446)"
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,"{'latitude': 34.0446, 'longitude': -118.25444}",$$,344,34.044600,-118.254440,"(34.0446, -118.25444)"
3,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,"{'latitude': 34.0434989929199, 'longitude': -1...",$$,118,34.043499,-118.250206,"(34.0434989929199, -118.250205993652)"
4,Extra Special Delivery,False,https://www.yelp.com/biz/extra-special-deliver...,4.5,"{'latitude': 34.167, 'longitude': -118.37364}",$,130,34.167000,-118.373640,"(34.167, -118.37364)"


In [ ]:
df_lojas.drop('coordinates', axis = 1, inplace = True)

In [38]:
df_lojas.shape

(326, 9)

In [37]:
# We preview a sample of data
df_lojas.head()

,name,is_closed,url,rating,price,review_count,latitude,longitude,location
0,California Caregivers Alliance,False,https://www.yelp.com/biz/california-caregivers...,4.5,NaN,299,34.082350,-118.272037,"(34.08235, -118.272037)"
1,Herbarium,False,https://www.yelp.com/biz/herbarium-west-hollyw...,4.5,$$,289,34.088530,-118.344600,"(34.08853, -118.3446)"
2,MedMen Los Angeles - DTLA,False,https://www.yelp.com/biz/medmen-los-angeles-dt...,4.0,$$,344,34.044600,-118.254440,"(34.0446, -118.25444)"
3,HERB,False,https://www.yelp.com/biz/herb-los-angeles-3?ad...,4.5,$$,118,34.043499,-118.250206,"(34.0434989929199, -118.250205993652)"
4,Extra Special Delivery,False,https://www.yelp.com/biz/extra-special-deliver...,4.5,$,130,34.167000,-118.373640,"(34.167, -118.37364)"


Por hora nosso primeiro dataset está concluído. Vamos salvá-lo em disco.

In [ ]:
# Save the stores dataframe to a csv file
df_lojas.to_csv('dados/df_lojas.csv')

<a href="https://github.com/rodolffoterra/Projct_Marijuana_Legalization_TS/blob/main/Projct_Marijuana_Legalization_TS_Part2.ipynb">Continuação do Projeto: Parte. 2</a>

![Rodolfo Terra](https://raw.githubusercontent.com/rodolffoterra/Projct_Marijuana_Legalization_TS/main/imagnes/logo.png)